In [1]:
import os, re
from bs4 import BeautifulSoup
from selenium import webdriver
import numpy as np
#import requests #request+bs4 조합만으로도 crawling가능
from selenium.webdriver.common.keys import Keys
import time
#import re
import getpass
import copy
import pandas as pd
import openpyxl


In [3]:
a=None
print(type(a))

<class 'NoneType'>


In [2]:
# 크롬드라이버
chromedriver = r"D:\Shin\Develop\python\NLP\nlp4DL\Crawling\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(chromedriver)

# 엑셀로 만들기

wb = openpyxl.Workbook()

# 로그인
id_ = 'siinwoo'
pw_ = 'woo960306'
# lec_nme = input()
base_url = 'https://everytime.kr'

driver.get(base_url + '/timetable')
driver.find_element_by_name("userid").send_keys(id_)
driver.find_element_by_name("password").send_keys(pw_)
driver.find_element_by_tag_name("input").send_keys(Keys.RETURN)

# time.sleep(2)
# driver.find_element_by_name("keyword").send_keys(lec_nme)
# driver.find_element_by_class_name("submit").send_keys(Keys.RETURN)
time.sleep(3)
while True:
    try:
        driver.find_element_by_xpath("""//*[@id="sheet"]/ul/li[3]/a""").click()
        time.sleep(2)
        break
    except:
        continue
time.sleep(3)

# 강의명 수집
lecture = []
kyoyang = []
while True:
    try:
        driver.find_element_by_xpath("""//*[@id="container"]/ul/li[1]""").click()
        time.sleep(2)
        break
    except:
        continue
while True:
    try:
        driver.find_element_by_xpath("""//*[@id="subjects"]/div[1]/a[4]""").click()
        time.sleep(2)
        break
    except:
        continue
while True:
    try:
        driver.find_element_by_xpath("""//*[@id="subjectCategoryFilter"]/div/ul/li[2]""").click()
        time.sleep(2)
        break
    except:
        continue
page = driver.page_source
soup = BeautifulSoup(page, "html.parser")
tmps = soup.find('ul','category').find('ul', 'unfolded').find_all('li')
for li in tmps:
    li = str(li)
    lec = re.findall(r"<li class=\"child\">(.*)</li>", li)
    kyoyang.append(lec[0])




for i in range(17):
    # while i == 0:
    #     try:
    #         driver.find_element_by_xpath("""//*[@id="subjectCategoryFilter"]/div/ul/li[2]""").click()
    #         time.sleep(2)
    #         break
    #     except:
    #         continue
    while True:
        try:
            driver.find_element_by_xpath("""//*[@id="subjectCategoryFilter"]/div/ul/ul[2]/li["""+str((i+1))+"""]""").click()
            time.sleep(2)
            break
        except:
            continue
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    tmps = soup.find('div','list').find('tbody').find_all('td')
    for td in tmps:
        td = str(td)
        if re.match(r"<td class=\"bold\">(.*)</td>", td):
            lec = re.findall(r"<td class=\"bold\">(.*)</td>", td)
            lecture.append(lec[0])
    while True:
        try:
            driver.find_element_by_xpath("""//*[@id="subjects"]/div[1]/a[4]""").click()
            time.sleep(2)
            break
        except:
            continue

_, indexes = np.unique(lecture, return_index=True)
lectures = [lecture[i] for i in np.sort(indexes)]

print(lectures)

lecture = []
for lect in lectures:
    lect = re.sub(r"(HUFS Career Vision Mentoring)\(진로\.취\/창업경력개발\)", r"\1", lect)
    lecture.append(lect)


print(lecture)


while True:
    try:
        driver.find_element_by_xpath("""//*[@id="subjectCategoryFilter"]/a""").click()
        time.sleep(1)
        break
    except:
        continue
lec_eval = []
for name in lecture:
    while True:
        try:
            driver.find_element_by_xpath("""//*[@id="menu"]/li[3]/a""").click()
            time.sleep(0.5)
            break
        except:
            continue

    while True:
        try:
            driver.find_element_by_name("keyword").send_keys(name)
            time.sleep(0.5)
            driver.find_element_by_xpath("""//*[@id="container"]/form/input[2]""").click()
            time.sleep(0.5)
            break
        except:
            continue
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    tmp_div = soup.find('div','lectures')
    if tmp_div == None:
        pass
    else:
        tmp = tmp_div.find_all('a')
        temp_lec = []
        if tmp == None:
            break
        else:
            cnt_sme_lec = len(tmp)
            for i in range(cnt_sme_lec):
                while True:
                    try:
                        driver.find_element_by_xpath("""//*[@id="container"]/div/a["""+str((i+1))+"""]""").click()
                        time.sleep(0.5)
                        break
                    except:
                        continue
                page = driver.page_source
                soup = BeautifulSoup(page, "html.parser")
                pf_tmps = soup.find('div', 'side head')
                if pf_tmps != None:
                    pf_tmp = pf_tmps.find_all('p')
                    for pf in pf_tmp:
                        pf = str(pf)
                        if re.match(r'<p><label>교수명</label><span>(.*)</span></p>', pf):
                            nme = re.findall(r'<p><label>교수명</label><span>(.*)</span></p>', pf)
                    tmp = soup.find('div','articles')
                    if tmp == None:
                        break
                    else:
                        tmps = tmp.find_all('article')
                        temp_eval = []
                        for article in tmps:
                            article = str(article)
                            set_eval = []
                            sem = re.findall(r"<span class=\"semester\">(.*)<\/span>", article)
                            lec = re.findall(r"<p class=\"text\">(.*)<\/p><\/article", article)
                            set_eval.append(name)
                            set_eval.append(nme[0])
                            set_eval.append(sem[0])
                            set_eval.append(lec[0])
                            temp_eval.append(set_eval)
                        temp_lec.append(temp_eval)
                    while True:
                        try:
                            driver.find_element_by_xpath("""//*[@id="menu"]/li[3]/a""").click()
                            time.sleep(0.5)
                            break
                        except:
                            continue

                    while True:
                        try:
                            driver.find_element_by_name("keyword").send_keys(name)
                            time.sleep(0.5)
                            driver.find_element_by_xpath("""//*[@id="container"]/form/input[2]""").click()
                            time.sleep(0.5)
                            break
                        except:
                            continue
                lec_eval.append(temp_lec)

print(lec_eval[:2])

['대학독일어2', '대학러시아어2', '대학말레이.인도네시아어2', '대학베트남어2', '대학스페인어2', '대학아랍어2', '대학영어2', '미네르바인문(1)읽기와쓰기-인간과문명', '미네르바인문(2)읽기와토의.토론-인간의삶과가치', '정보사회의통계활용', '컴퓨터프로그래밍', '컴퓨팅사고', '바둑', '스포츠의글로벌문화', '운동과건강', '웰빙시대의운동과다이어트', '체육(골프)', '체육(농구)', '체육(댄스스포츠)', '체육(동계스포츠)', '체육(레크리에이션)', '체육(스킨스쿠버)', '체육(승마)', '체육(야영생활과리더쉽)', '체육(여성호신술)', '체육(요가)', '체육(윈드서핑)', '체육(축구)', '체육(탁구)', '체육(태권도)', '체육(휘트니스)', '고전그리스어와그리스문명', '공감을위한 Lectio (TRTL 202)', '교양한자와한문', '라틴어와로마문명', '루쉰과동아시아문학과문화', '문예창작의이론과실제(교양)', '문학과대중', '문학과신화', '문학과영화', '문학을통한치유', '문학이야기', '서양명작의이해와감상', '설득과소통의레토릭', '언어와문화(교양)', '언어와인지', '언어와현대미디어사회', '언어학과심리실험', '에스페란토어의이해', '외국어로서의한국어교육의이해', '위대한개츠비로본문학비평이론', '유럽학,유럽문화와언어', '인문적사유와글쓰기', '인문학기반의데이터과학101:컴퓨터적사고와파이썬', '인문학명저산책', '인문학의이해-문학,역사,철학', '창의적독서와자기발견', '통번역의이론과실제', '한시로읽는동양의삶과미학', '희곡과시나리오작법', '히브리어와유대문화', '감성을위한 Lectio (TRTL 102)', '공연예술의이해와감상', '나를찾아떠나는크리에이티브드라마', '남아시아문화의이해', '다문화사회의이해', '다양성과공존의시각으로다시보는발칸-동유럽', '대중문화예술의이해', '도시와환경에대한예술적이해', '도시의미학과문화', '동남아시아문화의이해', '동아시아와일본문화', '동아시아

In [3]:
lec_nme = []
pf_nme = []
sem = []
eval_ = []
for lec_info in lec_eval:
    for lec in lec_info:
        for data in lec:
            lec_nme.append(data[0])
            pf_nme.append(data[1])
            sem.append(data[2])
            eval_.append(data[-1])

print(lec_nme[:5])
print(pf_nme[:5])
print(sem[:5])
print(eval_[:5])

['대학독일어2', '대학독일어2', '대학독일어2', '대학독일어2', '대학독일어2']
['김이천', '김이천', '김이천', '김이천', '김이천']
['19년 2학기 수강자', '19년 2학기 수강자', '19년 2학기 수강자', '18년 2학기 수강자', '18년 2학기 수강자']
['출결은 매주 과제 바탕으로 나옵니다! 나름 독일어 되게 열심히 했고, 많이 안 다고 생각했음에도 비쁠이네요. 성적 따기는 조금 힘든 것 같아요. 다만 독일어에 재미 붙이기에는 좋은 강의입니다!', '좋긴 하지만 과제가 은근 귀찮 교수님은 좋아요 근데 교양 외국어 학점 잘 받을 생각은 하지 마세요...', '과제는 매주 나오는데 복습한다고 생각하면 그리 어렵지 않아요! 시험도 생각보다 어렵지 않아요!!  그리고 교수님이 학습내용 반복해서 알려주셔서 공부할 때 편해요:)', '교수님이 수업시간 중에 시험에 나온다고 하는 부분 잘 체크하시고 교안 거의 외울정도로 시험치시면 점수 잘 나올거예요. 시험이 그렇게 어렵지 않아서 할만 했습니다. 수업도 늦게 시작해서 일찍 끝내주시고 과제도 학기 끝나기 전까지만 제출하면 점수 받습니다. 교수님이 너무 좋으셨어요.', '매주 과제가 있어서 조금 귀찮긴한데 나중에 시험문제로 와서 예습과 복습차원에서 정말 도움 많이 됐어요! 수업하시면서 시험에 나오는거 찝어주셔서 수업시간에 집중하는갈 추천드려요! 지각을 해도 상관없고 수업을 늦게 시작하시고 일찍 끝내주셔서 좋았어요 바로 연강이여도 충분히 다음 수업에 도착할 수 있어요! 정말 출석열심히하고 과제 꾸준히 내시고 교수님이 찍어주시는거 잘들으시면 에이쁠 충분히 나와요!😊']


In [4]:
df = pd.DataFrame(data={"강의 이름" : lec_nme, "교수" : pf_nme, "수강 학기" : sem, "강의평" : eval_})

df.head()

,강의 이름,교수,수강 학기,강의평
0,대학독일어2,김이천,19년 2학기 수강자,"출결은 매주 과제 바탕으로 나옵니다! 나름 독일어 되게 열심히 했고, 많이 안 다고..."
1,대학독일어2,김이천,19년 2학기 수강자,좋긴 하지만 과제가 은근 귀찮 교수님은 좋아요 근데 교양 외국어 학점 잘 받을 생각...
2,대학독일어2,김이천,19년 2학기 수강자,과제는 매주 나오는데 복습한다고 생각하면 그리 어렵지 않아요! 시험도 생각보다 어렵...
3,대학독일어2,김이천,18년 2학기 수강자,교수님이 수업시간 중에 시험에 나온다고 하는 부분 잘 체크하시고 교안 거의 외울정도...
4,대학독일어2,김이천,18년 2학기 수강자,매주 과제가 있어서 조금 귀찮긴한데 나중에 시험문제로 와서 예습과 복습차원에서 정말...


In [5]:
df.to_csv(r"D:\Shin\Develop\python\NLP\nlp4DL\Crawling\강의평.csv", header=True, encoding='utf-8')

In [37]:
lec = {}
for lec_info in lec_eval:
    lec['강의 이름'] = lec_info[0]
    for i in range(len(lec_info)):
        if i % 2 == 1:
            lec['교수명'] = lec_info[1]
    if len(lec_info) > 1:
        for i in range(len(lec_info[-1])):
            lec['수강학기'] = lec_info[-1][i][0]
            lec['강의평'] = lec_info[-1][i][-1]
    else:
        lec['수강학기'] = "강의평이 없습니다."
        lec['강의평'] = "강의평이 없습니다."

print(lec)

{&#39;강의 이름&#39;: &#39;현대인의정신건강&#39;, &#39;교수명&#39;: &#39;홍선임&#39;, &#39;수강학기&#39;: &#39;19년 2학기 수강자&lt;/span&gt;&lt;span class=&quot;posvote&quot;&gt;3&#39;, &#39;강의평&#39;: &#39;사담 개많음 맨날 유튜브 신서유기 짤 대학원 이야기 함&lt;br/&gt;근데 거기서 시험 문제 나옴 ㅋㅋ&lt;br/&gt;교수님 말 많고 범위 많고 시험은 깐깐하고 수업은 꽉 채움&lt;br/&gt;교양이 교양이 아님 드랍 안 한 걸 후회함&lt;br/&gt;시험 답안 쓸 때 한톨도 틀리지 않고 그대로 써야 점수 줌&#39;}


In [4]:
###다시 로그인부터 해야함
# 크롬드라이버
chromedriver = r"D:\Shin\Develop\python\NLP\nlp4DL\Crawling\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(chromedriver)

# 로그인
id_ = 'siinwoo'
pw_ = 'woo960306'
# lec_nme = input()
base_url = 'https://everytime.kr'

driver.get(base_url + '/timetable')
driver.find_element_by_name("userid").send_keys(id_)
driver.find_element_by_name("password").send_keys(pw_)
driver.find_element_by_tag_name("input").send_keys(Keys.RETURN)

# time.sleep(2)
# driver.find_element_by_name("keyword").send_keys(lec_nme)
# driver.find_element_by_class_name("submit").send_keys(Keys.RETURN)
time.sleep(1)
while True:
    try:
        driver.find_element_by_xpath("""//*[@id="sheet"]/ul/li[3]/a""").click()
        time.sleep(2)
        break
    except:
        continue
time.sleep(1)
lecture = ['HUFS Career Vision Mentoring(진로.취/창업경력개발)', '대학스페인어2']
lec_eval = []
for name in lecture:
    while True:
        try:
            driver.find_element_by_xpath("""//*[@id="menu"]/li[3]/a""").click()
            time.sleep(2)
            break
        except:
            continue

    while True:
        try:
            driver.find_element_by_name("keyword").send_keys(name)
            time.sleep(1)
            driver.find_element_by_xpath("""//*[@id="container"]/form/input[2]""").click()
            time.sleep(1)
            break
        except:
            continue
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    
    tmp = soup.find('div','lectures').find_all('a')
    temp_lec = []
    # temp_lec.append(name)
    cnt_sme_lec = len(tmp)
    for i in range(cnt_sme_lec):
        while True:
            try:
                driver.find_element_by_xpath("""//*[@id="container"]/div/a["""+str((i+1))+"""]""").click()
                time.sleep(1)
                break
            except:
                continue
        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")
        contents = []
        pf_tmp = soup.find('div', 'side head').find_all('p')
        for pf in pf_tmp:
            pf = str(pf)
            if re.match(r'<p><label>교수명</label><span>(.*)</span></p>', pf):
                nme = re.findall(r'<p><label>교수명</label><span>(.*)</span></p>', pf)
                # temp_lec.append(nme[0])
        tmp = soup.find('div','articles')
        if tmp == None:
            continue
        else:
            tmps = tmp.find_all('article')
            temp_eval = []
            for article in tmps:
                article = str(article)
                set_eval = []
                sem = re.findall(r"<span class=\"semester\">(.*)<\/span>", article)
                lec = re.findall(r"<p class=\"text\">(.*)<\/p><\/article", article)
                set_eval.append(name)
                set_eval.append(nme[0])
                set_eval.append(sem[0])
                set_eval.append(lec[0])
                temp_eval.append(set_eval)
            temp_lec.append(temp_eval)
        while True:
            try:
                driver.find_element_by_xpath("""//*[@id="menu"]/li[3]/a""").click()
                time.sleep(2)
                break
            except:
                continue

        while True:
            try:
                driver.find_element_by_name("keyword").send_keys(name)
                time.sleep(1)
                driver.find_element_by_xpath("""//*[@id="container"]/form/input[2]""").click()
                time.sleep(1)
                break
            except:
                continue
        
    lec_eval.append(temp_lec)

print(lec_eval)



AttributeError: 'NoneType' object has no attribute 'find_all'

In [12]:
for lec in lec_eval:
    print(lec)

[[[&#39;대학독일어2&#39;, &#39;김이천&#39;, &#39;19년 2학기 수강자&#39;, &#39;출결은 매주 과제 바탕으로 나옵니다! 나름 독일어 되게 열심히 했고, 많이 안 다고 생각했음에도 비쁠이네요. 성적 따기는 조금 힘든 것 같아요. 다만 독일어에 재미 붙이기에는 좋은 강의입니다!&#39;], [&#39;대학독일어2&#39;, &#39;김이천&#39;, &#39;19년 2학기 수강자&#39;, &#39;좋긴 하지만 과제가 은근 귀찮 교수님은 좋아요 근데 교양 외국어 학점 잘 받을 생각은 하지 마세요...&#39;], [&#39;대학독일어2&#39;, &#39;김이천&#39;, &#39;19년 2학기 수강자&#39;, &#39;과제는 매주 나오는데 복습한다고 생각하면 그리 어렵지 않아요! 시험도 생각보다 어렵지 않아요!!  그리고 교수님이 학습내용 반복해서 알려주셔서 공부할 때 편해요:)&#39;], [&#39;대학독일어2&#39;, &#39;김이천&#39;, &#39;18년 2학기 수강자&#39;, &#39;교수님이 수업시간 중에 시험에 나온다고 하는 부분 잘 체크하시고 교안 거의 외울정도로 시험치시면 점수 잘 나올거예요. 시험이 그렇게 어렵지 않아서 할만 했습니다. 수업도 늦게 시작해서 일찍 끝내주시고 과제도 학기 끝나기 전까지만 제출하면 점수 받습니다. 교수님이 너무 좋으셨어요.&#39;], [&#39;대학독일어2&#39;, &#39;김이천&#39;, &#39;18년 2학기 수강자&#39;, &#39;매주 과제가 있어서 조금 귀찮긴한데 나중에 시험문제로 와서 예습과 복습차원에서 정말 도움 많이 됐어요! 수업하시면서 시험에 나오는거 찝어주셔서 수업시간에 집중하는갈 추천드려요! 지각을 해도 상관없고 수업을 늦게 시작하시고 일찍 끝내주셔서 좋았어요 바로 연강이여도 충분히 다음 수업에 도착할 수 있어요! 정말 출석열심히하고 과제 꾸준히 내시고 교수님이 찍어주시는거 잘들으시면 에이쁠 충분히 나와요!😊&#39;], [&#39;대학독일어2&#3

In [ ]:
import requests
from bs4 import BeautifulSoup

def login(id_='siinwoo', pw_='woo960306'):
    logger_info = {'userid': id_, 'password': pw_, 'redirect': '/'}
    base_url = r"https://everytime.kr"

    with requests.Session() as sess:
        user_res = sess.post(base_url + '/user/login', data=logger_info)
        soup = BeautifulSoup(user_res.text, 'html.parser').select('body div')

        return True

prf